# Problema clasificación con RNA y Keras

In [61]:
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import random

import warnings

# Para ignorar todas las advertencias
warnings.filterwarnings("ignore")

In [62]:
from numpy.random import seed
seed(1)

In [63]:
# Cargamos el conjunto de datos
# os.chdir (os.getcwd())
df = pd.read_csv(r"data/diabetes.csv", sep=",", header=None)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       768 non-null    int64  
 1   1       768 non-null    int64  
 2   2       768 non-null    int64  
 3   3       768 non-null    int64  
 4   4       768 non-null    int64  
 5   5       768 non-null    float64
 6   6       768 non-null    float64
 7   7       768 non-null    int64  
 8   8       768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [5]:
df.head()

,0,1,2,3,4,5,6,7,8
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


Significado de las columnas:

 0. Number of times pregnant
 1. Plasma glucose concentration a 2 hours in an oral glucose tolerance test
 2. Diastolic blood pressure (mm Hg)
 3. Triceps skin fold thickness (mm)
 4. 2-Hour serum insulin (mu U/ml)
 5. Body mass index (weight in kg/(height in m)^2)
 6. Diabetes pedigree function
 7. Age (years)
 8. **Variable clase (Tiene diabetes)**  ———> VARIABLE OBJETIVO

In [6]:
# Dividimos los datos en X e y
X = df.iloc[:,0:8]
y = df.iloc[:,8]

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
                                        X,
                                        y,
                                        train_size   = 0.7,
                                        random_state = 123,
                                        shuffle      = True
                                    )

### Definimos el modelo Keras

In [8]:
# Para obtener todos el mismo resultado debemos añadir una semilla
tf.random.set_seed(1) 
np.random.seed(1)
random.seed(1)
seed(1)

In [9]:
# Definiremos el modelo como una secuencia de capas.
# Usaremos el modelo secuencial de manera que podamos ir añadiendo capas hasta estar contentos con la arquitectura desarrollada.
model = Sequential()

# Partimos de un sistema con 8 variables por lo que nuestra primera capa acomodará dichas variables
# En la primera capa oculta usaremos 12 neuronas y una función de activación ReLU
# En la segunda capa oculta usaremos 8 neuronas y una función de activación ReLU
# Finalmente en la de salida una neurona y función de activación sigmoide
model.add(Dense(12, activation='relu'))
# model.add(Dense(12, input_dim=8, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Nota: Fíjate que el total de neuronas de entrada, lo definimos en la primera capa con input_dim


### Compilamos el modelo

In [10]:
# La compilación usa (internamente) librerías numéricas muy eficientes como TensorFlow además de comprobar si tenemos GPU 
# o sólo CPU
# Es necesario definir la función de pérdida que vamos a minimizar (optimizar).  Para este caso minimizaremos 
# Binary Cross Entropy puesto que funciona bien para problemas binarios de clasificación.
# Como métrica (al ser clasificación) usaremos la precisión.
# Como optimizador, usaremos el algoritmo "adam" ya que ofrece buenos resultados en un amplio abanico de problemas y 
# además de manera rápida

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              #loss="binary_crossentropy", por que es una clasificación binaria
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=False, # usar cuando valores de predicción sean [0,1]
                                                      label_smoothing=0.0,
                                                      axis=-1,
                                                    #   reduction="auto", # ME DA ERROR si le pongo AUTO
                                                      name="binary_crossentropy"), 
              metrics=['accuracy']) # La métrica eleginda


In [11]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=tf.keras.losses.BinaryCrossentropy(
        from_logits=False,  # Usar cuando las predicciones están en el rango [0, 1]
        label_smoothing=0.0,
        axis=-1,
        reduction='sum_over_batch_size',  # Cambiado a un valor válido
        name="binary_crossentropy" 
    ),
    metrics=['accuracy']  # Métrica de precisión
)

### Entrenamos el modelo

In [12]:
# seed(1)
# random.set_seed(1)
# El argumento batch_size, nos permite definir el número de filas que se considerarán, antes de que los pesos del
# modelo se reajusten dentro de cada ciclo.
model.fit(X_train, y_train, epochs=150, batch_size=10)

# En el aprendizaje automático, entrenar modelos de aprendizaje profundo con grandes conjuntos de datos puede 
# requerir mucha memoria y recursos computacionales. El uso de un tamaño de lote adecuado puede ayudar a evitar 
# problemas de memoria y a mejorar la velocidad de entrenamiento.

# Por ejemplo, si tenemos un conjunto de datos de 1000 muestras y establecemos batch_size=10, esto significa 
# que se utilizarán 10 muestras a la vez para actualizar los pesos del modelo en cada iteración del entrenamiento. 
# Por lo tanto, el proceso de entrenamiento se dividirá en 100 iteraciones, una para cada lote de 10 muestras.

# El tamaño de lote adecuado depende del conjunto de datos y del modelo. En general, un tamaño de lote más grande
# puede proporcionar un mejor rendimiento de entrenamiento, pero requiere más memoria. Por otro lado, un tamaño de
# lote más pequeño puede proporcionar una mejor generalización del modelo, pero puede requerir más iteraciones para
# converger a una solución óptima.

# En resumen, batch_size es un parámetro importante en el proceso de entrenamiento de modelos de aprendizaje profundo en Keras y su elección puede afectar el rendimiento del modelo y la velocidad de entrenamiento.

Epoch 1/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.5780 - loss: 2.3916
Epoch 2/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6194 - loss: 1.0147
Epoch 3/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6078 - loss: 0.9235
Epoch 4/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6616 - loss: 0.8540
Epoch 5/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6732 - loss: 0.8054
Epoch 6/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6850 - loss: 0.7753
Epoch 7/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 998us/step - accuracy: 0.6898 - loss: 0.7419
Epoch 8/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 791us/step - accuracy: 0.6958 - loss: 0.7211
Epoch 9/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 988us/step - accuracy: 0.6951 - loss: 0.7082
Epoch 10/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 882us/step - accuracy: 0.6955 - loss: 0.6965
Epoch 11/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 988us/step - accuracy: 0.7058 - loss: 0.6852
Epoch 12/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step -

### Evaluamos el modelo

In [13]:
# Con la red neuronal entrenada, ahora debemos evaluar cómo ha funcionado.
_, accuracy = model.evaluate(X_train, y_train)
print('Accuracy: %.2f' % (accuracy*100))

17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7314 - loss: 0.5314
Accuracy: 75.98


In [14]:
# No es un mal resultado, tenemos una precisión de más del 75%

### Predicciones

In [15]:
predicciones = model.predict(X_test)

# La función sigmoide nos devueve los resultados en formato probabilidad.
# Convertimos los mismos a casos, tomando como umbral 0.5
y_pred = (predicciones > 0.5).astype(int)
y_pred

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 


array([[1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
    

In [16]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[123,  20],
       [ 48,  40]])

## Ejercicios

1.Configura la red neuronal para que trabaje con 3 capas. 
- La primera con 15 neuronas y función de activación sigmoide. 
- La segunda con 10 neuronas y función de activación sigmoide.
- La tercera con 8 neuronas y función de activación ReLU.
- Una capa de salida con 1 neurona y función de activación sigmoide.

In [17]:
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import random

In [18]:
# Cargamos el conjunto de datos
# os.chdir (os.getcwd())
df = pd.read_csv(r"data/diabetes.csv", sep=",", header=None)

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       768 non-null    int64  
 1   1       768 non-null    int64  
 2   2       768 non-null    int64  
 3   3       768 non-null    int64  
 4   4       768 non-null    int64  
 5   5       768 non-null    float64
 6   6       768 non-null    float64
 7   7       768 non-null    int64  
 8   8       768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [20]:
df.head()

,0,1,2,3,4,5,6,7,8
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [21]:
# Dividimos los datos en X e y
X = df.iloc[:,0:8]
y = df.iloc[:,8]

In [22]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
                                        X,
                                        y,
                                        train_size   = 0.7,
                                        random_state = 123,
                                        shuffle      = True
                                    )

In [23]:
tf.random.set_seed(1) 

In [24]:
# Definiremos el modelo como una secuencia de capas.
# Usaremos el modelo secuencial de manera que podamos ir añadiendo capas hasta estar contentos con la arquitectura desarrollada.
model = Sequential()

# Partimos de un sistema con 8 variables por lo que nuestra primera capa acomodará dichas variables
# En la primera capa oculta usaremos 15 neuronas y una función de activación Sigmoid
# En la primera capa oculta usaremos 10 neuronas y una función de activación Sigmoid
# En la segunda capa oculta usaremos 8 neuronas y una función de activación ReLU
# Finalmente en la de salida una neurona y función de activación sigmoide
model.add(Dense(15, activation='sigmoid'))
model.add(Dense(10, activation='sigmoid'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Nota: Fíjate que el total de neuronas de entrada, lo definimos en la primera capa con input_dim


In [25]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=tf.keras.losses.BinaryCrossentropy(
        from_logits=False,  # Usar cuando las predicciones están en el rango [0, 1]
        label_smoothing=0.0,
        axis=-1,
        reduction='sum_over_batch_size',  # Cambiado a un valor válido
        name="binary_crossentropy" 
    ),
    metrics=['accuracy']  # Métrica de precisión
)

Entrenamos modelo

In [26]:
# seed(1)
# random.set_seed(1)
# El argumento batch_size, nos permite definir el número de filas que se considerarán, antes de que los pesos del
# modelo se reajusten dentro de cada ciclo.
model.fit(X_train, y_train, epochs=150, batch_size=10)

# En el aprendizaje automático, entrenar modelos de aprendizaje profundo con grandes conjuntos de datos puede 
# requerir mucha memoria y recursos computacionales. El uso de un tamaño de lote adecuado puede ayudar a evitar 
# problemas de memoria y a mejorar la velocidad de entrenamiento.

# Por ejemplo, si tenemos un conjunto de datos de 1000 muestras y establecemos batch_size=10, esto significa 
# que se utilizarán 10 muestras a la vez para actualizar los pesos del modelo en cada iteración del entrenamiento. 
# Por lo tanto, el proceso de entrenamiento se dividirá en 100 iteraciones, una para cada lote de 10 muestras.

# El tamaño de lote adecuado depende del conjunto de datos y del modelo. En general, un tamaño de lote más grande
# puede proporcionar un mejor rendimiento de entrenamiento, pero requiere más memoria. Por otro lado, un tamaño de
# lote más pequeño puede proporcionar una mejor generalización del modelo, pero puede requerir más iteraciones para
# converger a una solución óptima.

# En resumen, batch_size es un parámetro importante en el proceso de entrenamiento de modelos de aprendizaje profundo en Keras y su elección puede afectar el rendimiento del modelo y la velocidad de entrenamiento.

Epoch 1/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.6572 - loss: 0.6479
Epoch 2/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6572 - loss: 0.6420
Epoch 3/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6572 - loss: 0.6366
Epoch 4/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6572 - loss: 0.6336
Epoch 5/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 991us/step - accuracy: 0.6572 - loss: 0.6315
Epoch 6/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6572 - loss: 0.6304
Epoch 7/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 972us/step - accuracy: 0.6572 - loss: 0.6284
Epoch 8/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6572 - loss: 0.6267
Epoch 9/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 794us/step - accuracy: 0.6599 - loss: 0.6252
Epoch 10/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6599 - loss: 0.6230
Epoch 11/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 861us/step - accuracy: 0.6596 - loss: 0.6223
Epoch 12/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 920us/step -

#### Evaluamos modelo

In [27]:
# Con la red neuronal entrenada, ahora debemos evaluar cómo ha funcionado.
_, accuracy = model.evaluate(X_train, y_train)
print('Accuracy: %.2f' % (accuracy*100))

17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 966us/step - accuracy: 0.7338 - loss: 0.5217
Accuracy: 73.93


#### Predicciones

In [28]:
predicciones = model.predict(X_test)

# La función sigmoide nos devueve los resultados en formato probabilidad.
# Convertimos los mismos a casos, tomando como umbral 0.5
y_pred = (predicciones > 0.5).astype(int)
y_pred

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step


array([[0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [1],
    

In [29]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[114,  29],
       [ 52,  36]])

2.Configura la red neuronal para que trabaje con las 3 mismas capas del ejemplo inicial, pero esta vez usa como función de activación de la capa de salida 'softmax'

In [30]:
# Definiremos el modelo como una secuencia de capas.
# Usaremos el modelo secuencial de manera que podamos ir añadiendo capas hasta estar contentos con la arquitectura desarrollada.
model = Sequential()

# Partimos de un sistema con 8 variables por lo que nuestra primera capa acomodará dichas variables
# En la primera capa oculta usaremos 15 neuronas y una función de activación Sigmoid
# En la primera capa oculta usaremos 10 neuronas y una función de activación Sigmoid
# En la segunda capa oculta usaremos 8 neuronas y una función de activación ReLU
# Finalmente en la de salida una neurona y función de activación sigmoide
model.add(Dense(15, activation='sigmoid'))
model.add(Dense(10, activation='sigmoid'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='softmax'))

# Nota: Fíjate que el total de neuronas de entrada, lo definimos en la primera capa con input_dim


In [31]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=tf.keras.losses.BinaryCrossentropy(
        from_logits=False,  # Usar cuando las predicciones están en el rango [0, 1]
        label_smoothing=0.0,
        axis=-1,
        reduction='sum_over_batch_size',  # Cambiado a un valor válido
        name="binary_crossentropy" 
    ),
    metrics=['accuracy']  # Métrica de precisión
)

In [32]:
# seed(1)
# random.set_seed(1)
# El argumento batch_size, nos permite definir el número de filas que se considerarán, antes de que los pesos del
# modelo se reajusten dentro de cada ciclo.
model.fit(X_train, y_train, epochs=150, batch_size=10)

# En el aprendizaje automático, entrenar modelos de aprendizaje profundo con grandes conjuntos de datos puede 
# requerir mucha memoria y recursos computacionales. El uso de un tamaño de lote adecuado puede ayudar a evitar 
# problemas de memoria y a mejorar la velocidad de entrenamiento.

# Por ejemplo, si tenemos un conjunto de datos de 1000 muestras y establecemos batch_size=10, esto significa 
# que se utilizarán 10 muestras a la vez para actualizar los pesos del modelo en cada iteración del entrenamiento. 
# Por lo tanto, el proceso de entrenamiento se dividirá en 100 iteraciones, una para cada lote de 10 muestras.

# El tamaño de lote adecuado depende del conjunto de datos y del modelo. En general, un tamaño de lote más grande
# puede proporcionar un mejor rendimiento de entrenamiento, pero requiere más memoria. Por otro lado, un tamaño de
# lote más pequeño puede proporcionar una mejor generalización del modelo, pero puede requerir más iteraciones para
# converger a una solución óptima.

# En resumen, batch_size es un parámetro importante en el proceso de entrenamiento de modelos de aprendizaje profundo en Keras y su elección puede afectar el rendimiento del modelo y la velocidad de entrenamiento.

Epoch 1/150


/Users/egoitzaulestiapadilla/opt/anaconda3/lib/python3.9/site-packages/keras/src/ops/nn.py:545: UserWarning: You are using a softmax over axis -1 of a tensor of shape (None, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


54/54 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.3428 - loss: 0.6696
Epoch 2/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 953us/step - accuracy: 0.3428 - loss: 0.6418
Epoch 3/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 802us/step - accuracy: 0.3423 - loss: 0.6389
Epoch 4/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 727us/step - accuracy: 0.3428 - loss: 0.6377
Epoch 5/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 892us/step - accuracy: 0.3423 - loss: 0.6355
Epoch 6/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 833us/step - accuracy: 0.3428 - loss: 0.6343
Epoch 7/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 820us/step - accuracy: 0.3428 - loss: 0.6326
Epoch 8/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 705us/step - accuracy: 0.3428 - loss: 0.6309
Epoch 9/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 721us/step - accuracy: 0.3428 - loss: 0.6291
Epoch 10/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 721us/step - accuracy: 0.3428 - loss: 0.6272
Epoch 11/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 792us/step - accuracy: 0.3428 - loss: 0.6251
Epoch 12/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 851us/step -

In [33]:
#### EVALUAMOS MODELO

# Con la red neuronal entrenada, ahora debemos evaluar cómo ha funcionado.
_, accuracy = model.evaluate(X_train, y_train)
print('Accuracy: %.2f' % (accuracy*100))

17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3470 - loss: 0.5721
Accuracy: 33.52


/Users/egoitzaulestiapadilla/opt/anaconda3/lib/python3.9/site-packages/keras/src/ops/nn.py:545: UserWarning: You are using a softmax over axis -1 of a tensor of shape (None, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


In [34]:
#### PREDICCIONES ####

predicciones = model.predict(X_test)

# La función sigmoide nos devueve los resultados en formato probabilidad.
# Convertimos los mismos a casos, tomando como umbral 0.5
y_pred = (predicciones > 0.5).astype(int)
y_pred

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step


/Users/egoitzaulestiapadilla/opt/anaconda3/lib/python3.9/site-packages/keras/src/ops/nn.py:545: UserWarning: You are using a softmax over axis -1 of a tensor of shape (32, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(
/Users/egoitzaulestiapadilla/opt/anaconda3/lib/python3.9/site-packages/keras/src/ops/nn.py:545: UserWarning: You are using a softmax over axis -1 of a tensor of shape (None, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


array([[1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
    

In [35]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[  0, 143],
       [  0,  88]])

### **3.En compile, cambia la configuración del optimizer, de manera que en vez de Adam, usemos esta vez SGD:**
optimizer=tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.1, nesterov=False, name='SGD')

In [36]:
# Definiremos el modelo como una secuencia de capas.
# Usaremos el modelo secuencial de manera que podamos ir añadiendo capas hasta estar contentos con la arquitectura desarrollada.
model = Sequential()

# Partimos de un sistema con 8 variables por lo que nuestra primera capa acomodará dichas variables
# En la primera capa oculta usaremos 15 neuronas y una función de activación Sigmoid
# En la primera capa oculta usaremos 10 neuronas y una función de activación Sigmoid
# En la segunda capa oculta usaremos 8 neuronas y una función de activación ReLU
# Finalmente en la de salida una neurona y función de activación sigmoide
model.add(Dense(15, activation='sigmoid'))
model.add(Dense(10, activation='sigmoid'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='softmax'))

# Nota: Fíjate que el total de neuronas de entrada, lo definimos en la primera capa con input_dim

In [37]:
model.compile(
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.1, nesterov=False, name='SGD'),
    loss=tf.keras.losses.BinaryCrossentropy(
        from_logits=False,  # Usar cuando las predicciones están en el rango [0, 1]
        label_smoothing=0.0,
        axis=-1,
        reduction='sum_over_batch_size',  # Cambiado a un valor válido
        name="SGD" 
    ),
    metrics=['accuracy']  # Métrica de precisión
)

In [38]:
# seed(1)
# random.set_seed(1)
# El argumento batch_size, nos permite definir el número de filas que se considerarán, antes de que los pesos del
# modelo se reajusten dentro de cada ciclo.
model.fit(X_train, y_train, epochs=150, batch_size=10)

# En el aprendizaje automático, entrenar modelos de aprendizaje profundo con grandes conjuntos de datos puede 
# requerir mucha memoria y recursos computacionales. El uso de un tamaño de lote adecuado puede ayudar a evitar 
# problemas de memoria y a mejorar la velocidad de entrenamiento.

# Por ejemplo, si tenemos un conjunto de datos de 1000 muestras y establecemos batch_size=10, esto significa 
# que se utilizarán 10 muestras a la vez para actualizar los pesos del modelo en cada iteración del entrenamiento. 
# Por lo tanto, el proceso de entrenamiento se dividirá en 100 iteraciones, una para cada lote de 10 muestras.

# El tamaño de lote adecuado depende del conjunto de datos y del modelo. En general, un tamaño de lote más grande
# puede proporcionar un mejor rendimiento de entrenamiento, pero requiere más memoria. Por otro lado, un tamaño de
# lote más pequeño puede proporcionar una mejor generalización del modelo, pero puede requerir más iteraciones para
# converger a una solución óptima.

# En resumen, batch_size es un parámetro importante en el proceso de entrenamiento de modelos de aprendizaje profundo en Keras y su elección puede afectar el rendimiento del modelo y la velocidad de entrenamiento.

Epoch 1/150


/Users/egoitzaulestiapadilla/opt/anaconda3/lib/python3.9/site-packages/keras/src/ops/nn.py:545: UserWarning: You are using a softmax over axis -1 of a tensor of shape (None, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.3428 - loss: 0.7085
Epoch 2/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3428 - loss: 0.6544
Epoch 3/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3428 - loss: 0.6434
Epoch 4/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3428 - loss: 0.6409  
Epoch 5/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3428 - loss: 0.6403
Epoch 6/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3422 - loss: 0.6394
Epoch 7/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3432 - loss: 0.6398
Epoch 8/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3415 - loss: 0.6384
Epoch 9/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3428 - loss: 0.6390
Epoch 10/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3416 - loss: 0.6379
Epoch 11/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3428 - loss: 0.6386
Epoch 12/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.3424 - 

In [39]:
#### EVALUAMOS MODELO

# Con la red neuronal entrenada, ahora debemos evaluar cómo ha funcionado.
_, accuracy = model.evaluate(X_train, y_train)
print('Accuracy: %.2f' % (accuracy*100))

17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3495 - loss: 0.6024
Accuracy: 33.52


/Users/egoitzaulestiapadilla/opt/anaconda3/lib/python3.9/site-packages/keras/src/ops/nn.py:545: UserWarning: You are using a softmax over axis -1 of a tensor of shape (None, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


In [40]:
#### PREDICCIONES ####

predicciones = model.predict(X_test)

# La función sigmoide nos devueve los resultados en formato probabilidad.
# Convertimos los mismos a casos, tomando como umbral 0.5
y_pred = (predicciones > 0.5).astype(int)
y_pred

/Users/egoitzaulestiapadilla/opt/anaconda3/lib/python3.9/site-packages/keras/src/ops/nn.py:545: UserWarning: You are using a softmax over axis -1 of a tensor of shape (32, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step


/Users/egoitzaulestiapadilla/opt/anaconda3/lib/python3.9/site-packages/keras/src/ops/nn.py:545: UserWarning: You are using a softmax over axis -1 of a tensor of shape (None, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


array([[1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
    

In [41]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[  0, 143],
       [  0,  88]])

3.1.Cambia ahora la función de activación de la capa de salida a sigmoide

In [57]:
# Definiremos el modelo como una secuencia de capas.
# Usaremos el modelo secuencial de manera que podamos ir añadiendo capas hasta estar contentos con la arquitectura desarrollada.
model = Sequential()

# Partimos de un sistema con 8 variables por lo que nuestra primera capa acomodará dichas variables
# En la primera capa oculta usaremos 15 neuronas y una función de activación Sigmoid
# En la primera capa oculta usaremos 10 neuronas y una función de activación Sigmoid
# En la segunda capa oculta usaremos 8 neuronas y una función de activación ReLU
# Finalmente en la de salida una neurona y función de activación sigmoide
model.add(Dense(15, activation='sigmoid'))
model.add(Dense(10, activation='sigmoid'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Nota: Fíjate que el total de neuronas de entrada, lo definimos en la primera capa con input_dim

In [43]:
model.compile(
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.1, nesterov=False, name='SGD'),
    loss=tf.keras.losses.BinaryCrossentropy(
        from_logits=False,  # Usar cuando las predicciones están en el rango [0, 1]
        label_smoothing=0.0,
        axis=-1,
        reduction='sum_over_batch_size',  # Cambiado a un valor válido
        name="SGD" 
    ),
    metrics=['accuracy']  # Métrica de precisión
)

In [44]:
# seed(1)
# random.set_seed(1)
# El argumento batch_size, nos permite definir el número de filas que se considerarán, antes de que los pesos del
# modelo se reajusten dentro de cada ciclo.
model.fit(X_train, y_train, epochs=150, batch_size=10)

# En el aprendizaje automático, entrenar modelos de aprendizaje profundo con grandes conjuntos de datos puede 
# requerir mucha memoria y recursos computacionales. El uso de un tamaño de lote adecuado puede ayudar a evitar 
# problemas de memoria y a mejorar la velocidad de entrenamiento.

# Por ejemplo, si tenemos un conjunto de datos de 1000 muestras y establecemos batch_size=10, esto significa 
# que se utilizarán 10 muestras a la vez para actualizar los pesos del modelo en cada iteración del entrenamiento. 
# Por lo tanto, el proceso de entrenamiento se dividirá en 100 iteraciones, una para cada lote de 10 muestras.

# El tamaño de lote adecuado depende del conjunto de datos y del modelo. En general, un tamaño de lote más grande
# puede proporcionar un mejor rendimiento de entrenamiento, pero requiere más memoria. Por otro lado, un tamaño de
# lote más pequeño puede proporcionar una mejor generalización del modelo, pero puede requerir más iteraciones para
# converger a una solución óptima.

# En resumen, batch_size es un parámetro importante en el proceso de entrenamiento de modelos de aprendizaje profundo en Keras y su elección puede afectar el rendimiento del modelo y la velocidad de entrenamiento.

Epoch 1/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 954us/step - accuracy: 0.3432 - loss: 0.8511
Epoch 2/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4290 - loss: 0.7035
Epoch 3/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6572 - loss: 0.6670
Epoch 4/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6577 - loss: 0.6557
Epoch 5/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6573 - loss: 0.6512
Epoch 6/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6572 - loss: 0.6481
Epoch 7/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6572 - loss: 0.6459
Epoch 8/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6568 - loss: 0.6445
Epoch 9/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6575 - loss: 0.6430
Epoch 10/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6573 - loss: 0.6420
Epoch 11/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6572 - loss: 0.6411
Epoch 12/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accurac

In [45]:
#### EVALUAMOS MODELO

# Con la red neuronal entrenada, ahora debemos evaluar cómo ha funcionado.
_, accuracy = model.evaluate(X_train, y_train)
print('Accuracy: %.2f' % (accuracy*100))

17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 893us/step - accuracy: 0.6572 - loss: 0.6145
Accuracy: 67.23


In [46]:
#### PREDICCIONES ####

predicciones = model.predict(X_test)

# La función sigmoide nos devueve los resultados en formato probabilidad.
# Convertimos los mismos a casos, tomando como umbral 0.5
y_pred = (predicciones > 0.5).astype(int)
y_pred

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step


array([[0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
    

In [47]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[143,   0],
       [ 86,   2]])

4.En compile, cambia la configuración del loss para usar como función de pérdida CategoricalCrossentropy.

tf.keras.losses.CategoricalCrossentropy(
    from_logits=False,
    label_smoothing=0.0,
    axis=-1,
    reduction="auto",
    name="categorical_crossentropy"),


In [49]:
model.compile(
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.1, nesterov=False, name='SGD'),
    loss = tf.keras.losses.CategoricalCrossentropy(
        from_logits=False,
        label_smoothing=0.0,
        axis=-1,
        # reduction="auto",
        name="categorical_crossentropy"
    ),
    metrics=['accuracy']  # Métrica de precisión
)

In [50]:
# seed(1)
# random.set_seed(1)
# El argumento batch_size, nos permite definir el número de filas que se considerarán, antes de que los pesos del
# modelo se reajusten dentro de cada ciclo.
model.fit(X_train, y_train, epochs=150, batch_size=10)

# En el aprendizaje automático, entrenar modelos de aprendizaje profundo con grandes conjuntos de datos puede 
# requerir mucha memoria y recursos computacionales. El uso de un tamaño de lote adecuado puede ayudar a evitar 
# problemas de memoria y a mejorar la velocidad de entrenamiento.

# Por ejemplo, si tenemos un conjunto de datos de 1000 muestras y establecemos batch_size=10, esto significa 
# que se utilizarán 10 muestras a la vez para actualizar los pesos del modelo en cada iteración del entrenamiento. 
# Por lo tanto, el proceso de entrenamiento se dividirá en 100 iteraciones, una para cada lote de 10 muestras.

# El tamaño de lote adecuado depende del conjunto de datos y del modelo. En general, un tamaño de lote más grande
# puede proporcionar un mejor rendimiento de entrenamiento, pero requiere más memoria. Por otro lado, un tamaño de
# lote más pequeño puede proporcionar una mejor generalización del modelo, pero puede requerir más iteraciones para
# converger a una solución óptima.

# En resumen, batch_size es un parámetro importante en el proceso de entrenamiento de modelos de aprendizaje profundo en Keras y su elección puede afectar el rendimiento del modelo y la velocidad de entrenamiento.

Epoch 1/150


/Users/egoitzaulestiapadilla/opt/anaconda3/lib/python3.9/site-packages/keras/src/losses/losses.py:27: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return self.fn(y_true, y_pred, **self._fn_kwargs)


54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 946us/step - accuracy: 0.6489 - loss: 0.0000e+00
Epoch 2/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6574 - loss: 0.0000e+00
Epoch 3/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6574 - loss: 0.0000e+00
Epoch 4/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6575 - loss: 0.0000e+00
Epoch 5/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6567 - loss: 0.0000e+00
Epoch 6/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6582 - loss: 0.0000e+00
Epoch 7/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6578 - loss: 0.0000e+00
Epoch 8/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6572 - loss: 0.0000e+00
Epoch 9/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6572 - loss: 0.0000e+00
Epoch 10/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6572 - loss: 0.0000e+00
Epoch 11/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6572 - loss: 0.0000e+00
Epoch 12/150
54/54 ━━━━━━━━━━

In [51]:
#### EVALUAMOS MODELO

# Con la red neuronal entrenada, ahora debemos evaluar cómo ha funcionado.
_, accuracy = model.evaluate(X_train, y_train)
print('Accuracy: %.2f' % (accuracy*100))

17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 994us/step - accuracy: 0.6530 - loss: 0.0000e+00
Accuracy: 66.48


/Users/egoitzaulestiapadilla/opt/anaconda3/lib/python3.9/site-packages/keras/src/losses/losses.py:27: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return self.fn(y_true, y_pred, **self._fn_kwargs)


In [52]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[143,   0],
       [ 86,   2]])

## **5.Prueba diferentes configuraciones a ver si consigues mejorar el resultado inicial.**

#### Creamos una función para construir y entrenar el modelo con configuraciones personalizadas. Luego, iteraremos sobre varias configuraciones.

In [58]:
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf
from sklearn.metrics import accuracy_score

# Función para construir y evaluar el modelo
def build_and_evaluate_model(hidden_layer_sizes, activation, learning_rate, batch_size, epochs):
    # Construir el modelo
    model = Sequential()
    model.add(Dense(hidden_layer_sizes[0], activation=activation, input_dim=8))
    for neurons in hidden_layer_sizes[1:]:
        model.add(Dense(neurons, activation=activation))
    model.add(Dense(1, activation='sigmoid'))  # Capa de salida

    # Compilar el modelo
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    # Entrenar el modelo
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0)

    # Evaluar el modelo en el conjunto de prueba
    y_pred = (model.predict(X_test) > 0.5).astype(int)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Config: {hidden_layer_sizes}, {activation}, LR: {learning_rate}, Batch: {batch_size}, Epochs: {epochs} -> Accuracy: {accuracy:.2f}")
    return accuracy


#### Probar diferentes configuraciones

Ahora probamos varias combinaciones de hiperparámetros para encontrar la mejor.


In [59]:
# Configuraciones para probar
hidden_layer_configs = [[12, 8], [16, 8, 4], [32, 16], [64, 32, 16]]
activations = ['relu', 'tanh']
learning_rates = [0.01, 0.001]
batch_sizes = [10, 32, 64]
epochs_list = [50, 100]

# Resultados
best_accuracy = 0
best_config = None

for hidden_layers in hidden_layer_configs:
    for activation in activations:
        for lr in learning_rates:
            for batch_size in batch_sizes:
                for epochs in epochs_list:
                    accuracy = build_and_evaluate_model(hidden_layers, activation, lr, batch_size, epochs)
                    if accuracy > best_accuracy:
                        best_accuracy = accuracy
                        best_config = (hidden_layers, activation, lr, batch_size, epochs)

print(f"Mejor configuración: {best_config} -> Accuracy: {best_accuracy:.2f}")


/Users/egoitzaulestiapadilla/opt/anaconda3/lib/python3.9/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
Config: [12, 8], relu, LR: 0.01, Batch: 10, Epochs: 50 -> Accuracy: 0.62


/Users/egoitzaulestiapadilla/opt/anaconda3/lib/python3.9/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
Config: [12, 8], relu, LR: 0.01, Batch: 10, Epochs: 100 -> Accuracy: 0.79


/Users/egoitzaulestiapadilla/opt/anaconda3/lib/python3.9/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


...

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
Config: [64, 32, 16], tanh, LR: 0.001, Batch: 32, Epochs: 100 -> Accuracy: 0.65


/Users/egoitzaulestiapadilla/opt/anaconda3/lib/python3.9/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
Config: [64, 32, 16], tanh, LR: 0.001, Batch: 64, Epochs: 50 -> Accuracy: 0.67


/Users/egoitzaulestiapadilla/opt/anaconda3/lib/python3.9/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
Config: [64, 32, 16], tanh, LR: 0.001, Batch: 64, Epochs: 100 -> Accuracy: 0.65
Mejor configuración: ([12, 8], 'relu', 0.01, 10, 100) -> Accuracy: 0.79
